# **Simple PyTorch Training pipeline to train using SnapMix Augmentation.** 

*References :* 
* https://arxiv.org/abs/2012.04846
* https://github.com/Shaoli-Huang/SnapMix


* V15 : Base version
* V17 : Added warmup epoch(1), reducing snapmix to 0.5, Moved back prop etc outside autocast(as suggested in docs)
* V18 : Increased lr of backbone and running for 10 epochs now

In [1]:
!pip install timm

     |████████████████████████████████| 549 kB 13.4 MB/s 
     |████████████████████████████████| 199 kB 57.8 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.1
    Uninstalling typing-extensions-3.7.4.1:
      Successfully uninstalled typing-extensions-3.7.4.1
  Attempting uninstall: packaging
    Found existing installation: packaging 20.1
    Uninstalling packaging-20.1:
      Successfully uninstalled packaging-20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bokeh 2.2.3 requires tornado>=5.1, but you have tornado 5.0.2 which is incompatible.
aiobotocore 1.1.2 requires botocore<1.17.45,>=1.17.44, but you have botocore 1.19.31 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' com

In [2]:
from __future__ import print_function, division
import random
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import timm
from torchvision import models as tvmodels
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

import albumentations as A
import numpy as np
import cv2
from sklearn.model_selection import GroupKFold, StratifiedKFold

In [3]:
from albumentations import Compose
from albumentations.pytorch import ToTensorV2

In [4]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
DATA_PATH = '../input/cassava-leaf-disease-classification/'
NUM_FOLDS = 5
bs = 32
# Running only 5 epochs to test (Train more offline ^_^)
EPOCHS = 10
sz = 512
SNAPMIX_ALPHA = 5.0
SNAPMIX_PCT = 0.5
GRAD_ACCUM_STEPS = 1
TIMM_MODEL = 'resnet50'

In [6]:
# TODO : Play around with SNAPMIX_PCT, SNAPMIX_ALPHA and EPOCHS to converge for max accuracy

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 1234
seed_everything(SEED)    

# Cassava Dataset

In [8]:
class CassavaDataset(Dataset):
    """Cassava dataset."""

    def __init__(self, dataframe, root_dir, transforms=None):
        """
        Args:
            dataframe (string): dataframe train/valid
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        super().__init__()
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.dataframe)
    
    def get_img_bgr_to_rgb(self, path):
        im_bgr = cv2.imread(path)
        im_rgb = im_bgr[:, :, ::-1]
        return im_rgb

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = os.path.join(self.root_dir,
                                self.dataframe.iloc[idx, 0])
        image = self.get_img_bgr_to_rgb(img_name)
        if self.transforms:
            image = self.transforms(image=image)['image']
        csv_row = self.dataframe.iloc[idx, 1:]
        sample = {
            'image': image, 
            'label': csv_row.label,
        }
        return sample

In [9]:
train_df = pd.read_csv(DATA_PATH + "train.csv")

# Transforms using albumentations

In [10]:
def train_transforms():
    return Compose([
            A.RandomResizedCrop(sz, sz),
            #A.Transpose(p=0.5),
            A.HorizontalFlip(p=0.5),
            #A.VerticalFlip(p=0.5),
            #A.ShiftScaleRotate(p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            A.RandomBrightness(limit=0.3, p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)


def valid_transforms():
    return Compose([
            A.Resize(sz, sz),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            A.RandomBrightness(limit=0.3, p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)

# Model (modified to support SnapMix)

In [11]:
class CassavaNet(nn.Module):
    def __init__(self):
        super().__init__()
        backbone = timm.create_model(TIMM_MODEL, pretrained=True)
        n_features = backbone.fc.in_features
        self.backbone = nn.Sequential(*backbone.children())[:-2]
        self.classifier = nn.Linear(n_features, 5)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))

    def forward_features(self, x):
        x = self.backbone(x)
        return x

    def forward(self, x):
        feats = self.forward_features(x)
        x = self.pool(feats).view(x.size(0), -1)
        x = self.classifier(x)
        return x, feats

# Metrics

In [12]:
def accuracy_metric(input, targs):
    return accuracy_score(targs.cpu(), input.cpu())

def print_scores(scores):
    kaggle_metric = np.average(scores)
    print("Kaggle Metric: %f" % (kaggle_metric))
    
    return kaggle_metric

In [13]:
from sklearn.metrics import precision_recall_fscore_support

def prec_recall_fscore(input, targs):
    scores = precision_recall_fscore_support(targs.cpu(), input.cpu(), average='macro')
    return {'precision': scores[0], 'recall': scores[1], 'fscore': scores[2], 'support': scores[3]} # return dictionary of 4 values

def print_prec_recall_fscore(scores):
    print("Precision Recall FScores Metric:")
    print(scores)
    
    return scores

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Checkpoint method

In [15]:
def checkpoint(model, optimizer, epoch, current_metric, best_metric, fold):
    print("Metric improved from %f to %f , Saving Model at Epoch #%d" % (best_metric, current_metric, epoch))
    ckpt = {
        'model': CassavaNet(),
        'state_dict': model.state_dict(),
        #'optimizer' : optimizer.state_dict(),  # Commenting this out to cheap out on space
        'metric': current_metric
    }
    torch.save(ckpt, 'ckpt_%s-%d-%d.pth' % (TIMM_MODEL, sz, fold))

# Create folds

In [16]:
folds = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, 
                        random_state=SEED).split(np.arange(train_df.shape[0]), 
                                                 train_df.label.values)

# SnapMix Augmentation

In [17]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def get_spm(input,target,model):
    imgsize = (sz, sz)
    bs = input.size(0)
    with torch.no_grad():
        output,fms = model(input)
        clsw = model.classifier
        weight = clsw.weight.data
        bias = clsw.bias.data
        weight = weight.view(weight.size(0),weight.size(1),1,1)
        fms = F.relu(fms)
        poolfea = F.adaptive_avg_pool2d(fms,(1,1)).squeeze()
        clslogit = F.softmax(clsw.forward(poolfea))
        logitlist = []
        for i in range(bs):
            logitlist.append(clslogit[i,target[i]])
        clslogit = torch.stack(logitlist)

        out = F.conv2d(fms, weight, bias=bias)

        outmaps = []
        for i in range(bs):
            evimap = out[i,target[i]]
            outmaps.append(evimap)

        outmaps = torch.stack(outmaps)
        if imgsize is not None:
            outmaps = outmaps.view(outmaps.size(0),1,outmaps.size(1),outmaps.size(2))
            outmaps = F.interpolate(outmaps,imgsize,mode='bilinear',align_corners=False)

        outmaps = outmaps.squeeze()

        for i in range(bs):
            outmaps[i] -= outmaps[i].min()
            outmaps[i] /= outmaps[i].sum()


    return outmaps,clslogit


def snapmix(input, target, alpha, model=None):

    r = np.random.rand(1)
    lam_a = torch.ones(input.size(0))
    lam_b = 1 - lam_a
    target_b = target.clone()

    if True:
        wfmaps,_ = get_spm(input, target, model)
        bs = input.size(0)
        lam = np.random.beta(alpha, alpha)
        lam1 = np.random.beta(alpha, alpha)
        rand_index = torch.randperm(bs).cuda()
        wfmaps_b = wfmaps[rand_index,:,:]
        target_b = target[rand_index]

        same_label = target == target_b
        bbx1, bby1, bbx2, bby2 = rand_bbox(input.size(), lam)
        bbx1_1, bby1_1, bbx2_1, bby2_1 = rand_bbox(input.size(), lam1)

        area = (bby2-bby1)*(bbx2-bbx1)
        area1 = (bby2_1-bby1_1)*(bbx2_1-bbx1_1)

        if  area1 > 0 and  area>0:
            ncont = input[rand_index, :, bbx1_1:bbx2_1, bby1_1:bby2_1].clone()
            ncont = F.interpolate(ncont, size=(bbx2-bbx1,bby2-bby1), mode='bilinear', align_corners=True)
            input[:, :, bbx1:bbx2, bby1:bby2] = ncont
            lam_a = 1 - wfmaps[:,bbx1:bbx2,bby1:bby2].sum(2).sum(1)/(wfmaps.sum(2).sum(1)+1e-8)
            lam_b = wfmaps_b[:,bbx1_1:bbx2_1,bby1_1:bby2_1].sum(2).sum(1)/(wfmaps_b.sum(2).sum(1)+1e-8)
            tmp = lam_a.clone()
            lam_a[same_label] += lam_b[same_label]
            lam_b[same_label] += tmp[same_label]
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (input.size()[-1] * input.size()[-2]))
            lam_a[torch.isnan(lam_a)] = lam
            lam_b[torch.isnan(lam_b)] = 1-lam

    return input,target,target_b,lam_a.cuda(),lam_b.cuda()

# SnapMix Criterion (Loss)

In [18]:
class SnapMixLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, criterion, outputs, ya, yb, lam_a, lam_b):
        loss_a = criterion(outputs, ya)
        loss_b = criterion(outputs, yb)
        loss = torch.mean(loss_a * lam_a + loss_b * lam_b)
        return loss

# ADDED: FOCAL LOSS

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

# Train & Validate

In [20]:
for fold_num, (train_split, valid_split) in enumerate(folds):
    train_set = train_df.iloc[train_split].reset_index(drop=True)
    valid_set = train_df.iloc[valid_split].reset_index(drop=True)
    
    train_ds = CassavaDataset(dataframe=train_set,
                          root_dir=DATA_PATH + 'train_images',
                          transforms=train_transforms())
    
    valid_ds = CassavaDataset(dataframe=valid_set,
                          root_dir=DATA_PATH + 'train_images',
                          transforms=valid_transforms())
    
    train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, 
                                           shuffle=True, num_workers=8, drop_last=True,
                                           pin_memory=True)
    valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs, 
                                           shuffle=False, num_workers=8, 
                                           pin_memory=True)
    
    losses = []
    batches = len(train_dl)
    val_batches = len(valid_dl)
    best_metric = 0
    
    model = CassavaNet().to(device)
    criterion = nn.CrossEntropyLoss(reduction='none').to(device)
    val_criterion = nn.CrossEntropyLoss().to(device)
    # criterion = FocalLoss().to(device)
    # val_criterion = FocalLoss().to(device)
    snapmix_criterion = SnapMixLoss().to(device)
    
    param_groups = [
       {'params': model.backbone.parameters(), 'lr': 1e-2},
       {'params': model.classifier.parameters()},
    ]
    optimizer = torch.optim.SGD(param_groups, lr=1e-1, momentum=0.9,
                                weight_decay=1e-4, nesterov=True)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1,20,40], 
                                                     gamma=0.1, last_epoch=-1, verbose=True)
    scaler = GradScaler()
    
    for epoch in range(EPOCHS):
        # ----------------- TRAINING  ----------------- 
        train_loss = 0
        progress = tqdm(enumerate(train_dl), desc="Loss: ", total=batches)

        model.train()
        for i, data in progress:
            image, label = data.values()
            X, y = image.to(device).float(), label.to(device).long()
            
            with autocast():
                
                rand = np.random.rand()
                if rand > (1.0-SNAPMIX_PCT):
                    X, ya, yb, lam_a, lam_b = snapmix(X, y, SNAPMIX_ALPHA, model)
                    outputs, _ = model(X)
                    loss = snapmix_criterion(criterion, outputs, ya, yb, lam_a, lam_b)
                    # loss = torch.mean(criterion(outputs, y))                        
                else:
                    outputs, _ = model(X)
                    loss = torch.mean(criterion(outputs, y))                
            scaler.scale(loss).backward()
            # Accumulate gradients
            if ((i + 1) % GRAD_ACCUM_STEPS == 0) or ((i + 1) == len(train_dl)):
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            train_loss += loss.item()
            cur_step = i+1
            trn_epoch_result = dict()
            trn_epoch_result['Epoch'] = epoch + 1
            # trn_epoch_result['FLtrain_loss'] = round(train_loss/cur_step, 4)
            trn_epoch_result['train_loss'] = round(train_loss/cur_step, 4)

            progress.set_description(str(trn_epoch_result))

        scheduler.step()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        # ----------------- VALIDATION  ----------------- 
        val_loss = 0
        scores = []

        model.eval()
        with torch.no_grad():
            for i, data in enumerate(valid_dl):
                image, label = data.values()
                X, y = image.to(device), label.to(device)
                outputs, _ = model(X)
                l = val_criterion(outputs, y)
                val_loss += l.item()

                preds = F.softmax(outputs).argmax(axis=1)
                scores.append(accuracy_metric(preds, y))                

        epoch_result = dict()
        epoch_result['Epoch'] = epoch + 1
        epoch_result['train_loss'] = round(train_loss/batches, 4)
        epoch_result['val_loss'] = round(val_loss/val_batches, 4)

        print(epoch_result)

        # Check if we need to save
        current_metric = print_scores(scores)
        # print_prec_recall_fscore(prec_recall_scores)
        
        if current_metric > best_metric:
            checkpoint(model, optimizer, epoch+1, current_metric, best_metric, fold_num)
            best_metric = current_metric
            
    del model, optimizer, train_dl, valid_dl, scaler, scheduler
    torch.cuda.empty_cache()
    
    # Train only a single fold
    break

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /root/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth


Adjusting learning rate of group 0 to 1.0000e-02.
Adjusting learning rate of group 1 to 1.0000e-01.


{'Epoch': 1, 'train_loss': 1.0332}: 100%|██████████| 534/534 [06:45<00:00,  1.32it/s]

Adjusting learning rate of group 0 to 1.0000e-03.
Adjusting learning rate of group 1 to 1.0000e-02.


{'Epoch': 1, 'train_loss': 1.0332, 'val_loss': 0.7142}
Kaggle Metric: 0.747590
Metric improved from 0.000000 to 0.747590 , Saving Model at Epoch #1


...

Adjusting learning rate of group 0 to 1.0000e-03.
Adjusting learning rate of group 1 to 1.0000e-02.


{'Epoch': 9, 'train_loss': 0.7702, 'val_loss': 0.5722}
Kaggle Metric: 0.795009


{'Epoch': 10, 'train_loss': 0.761}: 100%|██████████| 534/534 [06:39<00:00,  1.34it/s]

Adjusting learning rate of group 0 to 1.0000e-03.
Adjusting learning rate of group 1 to 1.0000e-02.


{'Epoch': 10, 'train_loss': 0.761, 'val_loss': 0.5585}
Kaggle Metric: 0.800218


In [21]:
print_prec_recall_fscore(prec_recall_fscore(preds, y))

Precision Recall FScores Metric:
{'precision': 0.825, 'recall': 0.8875, 'fscore': 0.830952380952381, 'support': None}


{'precision': 0.825,
 'recall': 0.8875,
 'fscore': 0.830952380952381,
 'support': None}